In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [2]:
df = pd.read_csv('email.csv')  # Replace with your dataset file
df['Label'] = df['Category'].map({'ham':0, "spam":1})
print(df.head)



train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['Message'], df['Label'], test_size = 0.2, random_state = 42
)



<bound method NDFrame.head of             Category                                            Message  Label
0                ham  Go until jurong point, crazy.. Available only ...    0.0
1                ham                      Ok lar... Joking wif u oni...    0.0
2               spam  Free entry in 2 a wkly comp to win FA Cup fina...    1.0
3                ham  U dun say so early hor... U c already then say...    0.0
4                ham  Nah I don't think he goes to usf, he lives aro...    0.0
...              ...                                                ...    ...
5568             ham               Will ü b going to esplanade fr home?    0.0
5569             ham  Pity, * was in mood for that. So...any other s...    0.0
5570             ham  The guy did some bitching but I acted like i'd...    0.0
5571             ham                         Rofl. Its true to its name    0.0
5572  {"mode":"full"                                    isActive:false}    NaN

[5573 rows x 3 column

In [3]:
# Tokenize Messages
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


#Tokenize the text data
train_encodings = tokenizer(list(train_texts), truncation = True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding = True, max_length=128)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
for batch in train_loader:
    print(batch['labels'].shape)  # Should be torch.Size([batch_size])
    print(batch['labels'])        # Should contain integers like [0, 1, 0, ...]
    break


NameError: name 'train_loader' is not defined

In [ ]:
for batch in train_loader:
    inputs = {key: val.to(device) for key, val in batch.items()}
    outputs = model(**inputs)
    print(outputs.logits.shape)  # Should be torch.Size([batch_size, num_labels])
    break


torch.Size([16, 2])


In [5]:
import torch
from transformers import BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader

# Dataset class
class MessageDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        # Sanitize labels
        try:
            self.labels = [int(label) for label in labels if str(label).isdigit() and 0 <= int(label) < 2]
        except ValueError:
            raise ValueError("Labels must be integers in the range [0, 1].")
        
        # Ensure label count matches encoding count
        if len(self.labels) != len(labels):
            raise ValueError("Mismatch between the number of valid labels and provided labels.")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Prepare encodings and labels for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Data Cleaning Function
def clean_labels(raw_labels):
    try:
        return [int(label) for label in raw_labels if str(label).isdigit() and 0 <= int(label) < 2]
    except ValueError:
        raise ValueError("Labels must be integers in the range [0, 1].")

train_labels_cleaned = clean_labels(train_labels)
val_labels_cleaned = clean_labels(val_labels)

# Create datasets
train_dataset = MessageDataset(train_encodings, train_labels_cleaned)
val_dataset = MessageDataset(val_encodings, val_labels_cleaned)

# Load the model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.config.problem_type = "single_label_classification"

# Set up data loaders
train_loader = DataLoader(train_dataset, batch_size=16)
val_loader = DataLoader(val_dataset, batch_size=16)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Training loop
for epoch in range(3):
    # Training
    model.train()
    for batch in train_loader:
        inputs = {key: val.to(device) for key, val in batch.items()}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    
    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**inputs)
            val_loss += outputs.loss.item()
    print(f"Epoch {epoch+1}, Validation Loss: {val_loss}")



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1, Validation Loss: 0
Epoch 2, Validation Loss: 0
Epoch 3, Validation Loss: 0


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [6]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from torch.optim import AdamW
from torch.utils.data import DataLoader

# Dataset class remains unchanged
class MessageDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = [int(label) for label in labels if str(label).isdigit() and 0 <= int(label) < 2]
        if len(self.labels) != len(labels):
            raise ValueError("Mismatch between valid labels and provided labels.")

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

# Load pre-trained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Example encodings and labels
texts = ["This is a great day.", "I feel sad."]
labels = [1, 0]

# Tokenize the data
encodings = tokenizer(texts, truncation=True, padding=True, max_length=512, return_tensors="pt")

# Create dataset and dataloader
dataset = MessageDataset(encodings, labels)
loader = DataLoader(dataset, batch_size=2)

# Load model and optimizer
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model.config.problem_type = "single_label_classification"
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

for epoch in range(3):
    model.train()
    for batch in loader:
        inputs = {key: val.to(device) for key, val in batch.items()}
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/hq/85dlj2ps64xd20knck25s6lh0000gn/T/ipykernel_1266/2145402956.py:18: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch 1, Loss: 0.806496798992157
Epoch 2, Loss: 0.5327339172363281
Epoch 3, Loss: 0.39364904165267944


In [11]:
import pandas as pd

# Load the dataset
df = pd.read_csv('email.csv')

# Extract texts and labels
texts = df['Text'].tolist()
labels = df['Label'].tolist()


KeyError: 'Text'